Import des modules et des bibliothèques Python

In [138]:
!pip install nltk

You should consider upgrading via the '/home/nidhal/miniconda3/bin/python -m pip install --upgrade pip' command.


In [139]:
!pip install -ignore-installed --upgrade tensorflow

Looking in indexes: gnore-installed
You should consider upgrading via the '/home/nidhal/miniconda3/bin/python -m pip install --upgrade pip' command.


In [140]:
import json
import string
import random

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download('omw-1.4')
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /home/nidhal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nidhal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nidhal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Séparation des données

In [121]:
data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Hello",
                "Bonjour",
                "Salam",
                "yo",
                "salut"
            ],
            "responses": [
                "Bonjour",
                "wa aleyk as salam wa rahmatulahi",
                "hello",
                "salutations !"
            ]
        },
        {
            "tag": "age",
            "patterns": [
                "Quel âge as-tu?",
                "C'est quand ton anniversaire?",
                "Quand es-tu né?"
            ],
            "responses": [
                "J'ai 25 ans",
                "Je suis né en 1996",
                "Ma dated'anniversaire est le 3 juillet et je suis né en 1996",
                "03/07/1996"
            ]
        },
        {
            "tag": "date",
            "patterns": [
                "Que fais-tu ce week-end?",
                "Tu veux qu'on fasse un truc ensemble?",
                "Quels sont tes plans pour cette semaine"
            ],
            "responses": [
                "Je suis libre toute la semaine",
                "Je n'ai rien de prévu",
                "Je ne suis pas occupé"
            ]
        },
        {
            "tag": "name",
            "patterns": [
                "Quel est ton prenom?",
                "Comment tu t'appelles?",
                "Qui es-tu?"
            ],
            "responses": [
                "Mon prénom est Miki",
                "Je suis Miki",
                "Miki"
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "bye",
                "Salut",
                "seenya",
                "adios",
                "cya"
            ],
            "responses": [
                "C'était sympa de te parler",
                "à plus tard",
                "On se reparle tres vite!"
            ]
        }
    ]
}

In [142]:
# initialization of lemmatizer to get the root of the words
lemmatizer = WordNetLemmatizer()

# list creation
words = []
classes = []
doc_X = []
doc_y = []

# browse with a For loop all intents
# tokenize each pattern and add the tokens to the words list, the patterns
# the tag associated with the intention are added to the corresponding lists
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])

    # add the tag to the classes if it is not already there
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

    # lemmatize all vocabulary words and convert them to lower case
    # if the words do not appear in the punctuation
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in
    string.punctuation]

    # sort the vocabulary and classes alphabetically and take the
    # set to make sure there are no duplicates
    words = sorted(set(words))
    classes = sorted(set(classes))

In [143]:
print(words)

['adios', 'anniversaire', 'as-tu', 'bonjour', 'bye', "c'est", 'ce', 'cette', 'comment', 'cya', 'ensemble', 'es-tu', 'est', 'fais-tu', 'fasse', 'hello', 'né', 'plan', 'pour', 'prenom', "qu'on", 'quand', 'que', 'quel', 'quels', 'qui', 'salam', 'salut', 'seenya', 'semaine', 'sont', "t'appelles", 'te', 'ton', 'truc', 'tu', 'un', 'veux', 'week-end', 'yo', 'âge']


In [144]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [145]:
print(doc_X)

['Hello', 'Bonjour', 'Salam', 'yo', 'salut', 'Quel âge as-tu?', "C'est quand ton anniversaire?", 'Quand es-tu né?', 'Que fais-tu ce week-end?', "Tu veux qu'on fasse un truc ensemble?", 'Quels sont tes plans pour cette semaine', 'Quel est ton prenom?', "Comment tu t'appelles?", 'Qui es-tu?', 'bye', 'Salut', 'seenya', 'adios', 'cya']


In [146]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


In [147]:
# liste pour les données d'entrainement
training = []
out_empty = [0] * len(classes)

# création du modèle d'ensemble de mots
for idx, doc in enumerate(doc_X) :
    bow = []
    text = lemmatizer.lemmatize(doc.lower() )
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    
    # marque l'index de la classe à laquelle le pattern atquel est associé à
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
   
    # ajoute le one hot encoded Bow et les classes associées à la liste training
    training.append([bow, output_row])

# mélanger les données et les convertir en array
random.shuffle(training)
training = np.array(training, dtype=object)

# séparer les features et les labels target
train_X = np.array(list(training[:,0]))
train_y = np.array(list(training[:,1]))

In [150]:
# definition de quelques paramètres
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200

In [151]:
# modèle Deep Learning

model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation="softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [152]:
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               5376      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 5)                 325       
                                                                 
Total params: 13,957
Trainable params: 13,957
Non-trainable params: 0
_________________________________________________________________
None


In [153]:
# entrainement du modèle
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 1.6703 - accuracy: 0.2632
Epoch 2/200
1/1 [==============================] - 0s 11ms/step - loss: 1.5303 - accuracy: 0.3684
Epoch 3/200
1/1 [==============================] - 0s 10ms/step - loss: 1.5150 - accuracy: 0.3158
Epoch 4/200
1/1 [==============================] - 0s 13ms/step - loss: 1.3872 - accuracy: 0.5789
Epoch 5/200
1/1 [==============================] - 0s 13ms/step - loss: 1.2862 - accuracy: 0.8421
Epoch 6/200
1/1 [==============================] - 0s 14ms/step - loss: 1.2088 - accuracy: 0.6316
Epoch 7/200
1/1 [==============================] - 0s 13ms/step - loss: 1.0988 - accuracy: 0.8421
Epoch 8/200
1/1 [==============================] - 0s 19ms/step - loss: 1.0388 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 19ms/step - loss: 0.8988 - accuracy: 0.7895
Epoch 10/200
1/1 [==============================] - 0s 14ms/step - loss: 0.7559 - accuracy: 0.9474
Epoch 11/200
1/1 [===

Epoch 84/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0552 - accuracy: 1.0000
Epoch 85/200
1/1 [==============================] - 0s 8ms/step - loss: 0.1048 - accuracy: 0.9474
Epoch 86/200
1/1 [==============================] - 0s 8ms/step - loss: 0.1168 - accuracy: 0.9474
Epoch 87/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0904 - accuracy: 0.9474
Epoch 88/200
1/1 [==============================] - 0s 9ms/step - loss: 0.1095 - accuracy: 0.9474
Epoch 89/200
1/1 [==============================] - 0s 10ms/step - loss: 0.1050 - accuracy: 0.9474
Epoch 90/200
1/1 [==============================] - 0s 8ms/step - loss: 0.0634 - accuracy: 0.9474
Epoch 91/200
1/1 [==============================] - 0s 10ms/step - loss: 0.0602 - accuracy: 0.9474
Epoch 92/200
1/1 [==============================] - 0s 7ms/step - loss: 0.1488 - accuracy: 0.9474
Epoch 93/200
1/1 [==============================] - 0s 8ms/step - loss: 0.1037 - accuracy: 0.8947
Epoch 94/200
1/1 [

1/1 [==============================] - 0s 10ms/step - loss: 0.0932 - accuracy: 0.9474
Epoch 167/200
1/1 [==============================] - 0s 11ms/step - loss: 0.0666 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 9ms/step - loss: 0.0654 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 9ms/step - loss: 0.0804 - accuracy: 0.9474
Epoch 170/200
1/1 [==============================] - 0s 9ms/step - loss: 0.2016 - accuracy: 0.8947
Epoch 171/200
1/1 [==============================] - 0s 8ms/step - loss: 0.0599 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 10ms/step - loss: 0.0747 - accuracy: 0.9474
Epoch 173/200
1/1 [==============================] - 0s 11ms/step - loss: 0.0666 - accuracy: 0.9474
Epoch 174/200
1/1 [==============================] - 0s 9ms/step - loss: 0.0896 - accuracy: 0.8947
Epoch 175/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0848 - accuracy: 0.8947
Epoch 176/200
1/1 [

In [155]:
def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(text, vocab):
    tokens = clean_up_sentences(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w :
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
    
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [ ]:
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

salam aleyk
1/1 [==============================] - 0s 27ms/step
wa aleyk as salam wa rahmatulahi
comment tu t'appelles ?
1/1 [==============================] - 0s 32ms/step
Mon prénom est Miki
on se voit ce week-end ?
1/1 [==============================] - 0s 30ms/step
Je ne suis pas occupé
ok à plus poto
1/1 [==============================] - 0s 29ms/step
à plus tard
